
## Toxic Comment Classification Challenge
<b>Authors:</b> Yohannis Telila, Dawit Anelay<br>
<b>Date :</b> 12/04/2021<br>
<b>Course:</b> Human Language Technologies, 2020/2021 A.Y<br>
<b>Lecturer:</b> Prof. Giuseppe Attardi<br>
<b>data set source :</b> https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data


## 1. Preprocessing and EDA
### 1.1 Understanding data
// Detailed comment coming up . . .

In [90]:
# importing necessary libraries
import pandas as pd
import numpy as np

In [104]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
testLabel = pd.read_csv('data/test_labels.csv')
sample = pd.read_csv('data/sample_submission.csv')

In [105]:
train.head(70)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
65,0028d62e8a5629aa,All of my edits are good. Cunts like you who ...,1,0,1,0,1,0
66,00290e2a171dd073,"""\n - NeilN's bang on - You aren't being haras...",0,0,0,0,0,0
67,002918ae66cc4bc2,"I went there around the same time he did, and ...",0,0,0,0,0,0
68,0029541a38c523a0,There must be some chemical imbalance in your ...,0,0,0,0,0,0


In [106]:
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [107]:
testLabel.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [108]:
train.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [109]:
train.shape

(159571, 8)

In [110]:
train.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [111]:
# getting numeric counts in a comment.
train['numerics_count'] = train['comment_text'].apply(lambda x: len([t for t in x.split() if t.isdigit()]))

In [112]:
train.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate,numerics_count
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805,0.322314
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420,1.662930
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,243.000000


### 1.2 Cleaning 

// Detailed comment coming up . . .


In [113]:
# Lower case conversion, this lambda function will convert the comments to a lower case.
train['comment_text'] = train['comment_text'].apply(lambda x: x.lower())
test['comment_text'] = test['comment_text'].apply(lambda x: x.lower())

In [114]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,numerics_count
0,0000997932d777bf,explanation\nwhy the edits made under my usern...,0,0,0,0,0,0,0
1,000103f0d9cfb60f,d'aww! he matches this background colour i'm s...,0,0,0,0,0,0,1
2,000113f07ec002fd,"hey man, i'm really not trying to edit war. it...",0,0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nmore\ni can't make any real suggestions on ...",0,0,0,0,0,0,0
4,0001d958c54c6e35,"you, sir, are my hero. any chance you remember...",0,0,0,0,0,0,0


### 1.3 Expanding contracted comments.

// Detailed comment coming up . . .


In [115]:
import json
# contracted words list with their expanded form.
with open('expandedContractions.json') as json_file:
    cList = json.load(json_file)

In [116]:
def expandComment(x):
    if type(x) is str:
        for key in cList:
            value = cList[key]
            x = x.replace(key, value)
        return x
    else:
        return x

In [117]:
train['comment_text'] = train['comment_text'].apply(lambda x: expandComment(x))
test['comment_text'] = test['comment_text'].apply(lambda x: expandComment(x))

In [118]:
train.head(50)["comment_text"][0]


'explanation\nwhy the edits made under my username hardcore metallica fan were reverted? they were not vandalisms, just closure on some gas after i voted at new york dolls fac. and please do not remove the template from the talk page since i am retired now.89.205.38.27'

### 1.4 Removing Stop Words (or/and Frequent words/ Rare words):
Filtering out unnecessary words. <br>
// More detail comments are coming up.

In [119]:
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [120]:
train['comment_text']

0         explanation\nwhy the edits made under my usern...
1         d'aww! he matches this background colour i am ...
2         hey man, i am really not trying to edit war. i...
3         "\nmore\ni cannot make any real suggestions on...
4         you, sir, are my hero. any chance you remember...
                                ...                        
159566    ":::::and for the second time of asking, when ...
159567    you should be ashamed of yourself \n\nthat is ...
159568    spitzer \n\numm, theres no actual article for ...
159569    and it looks like it was actually you who put ...
159570    "\nand ... i really do not think you understan...
Name: comment_text, Length: 159571, dtype: object

In [121]:
train['comment_text'] = train['comment_text'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop))
test['comment_text'] = test['comment_text'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop))

In [98]:
train['comment_text']

0         explanation why edits made username hardcore m...
1         d'aww! he matches background colour i'm seemin...
2         hey man, i'm really trying edit war. it's guy ...
3         " more i can't make real suggestions improveme...
4         you, sir, hero. any chance remember page that'...
                                ...                        
159566    ":::::and second time asking, view completely ...
159567    you ashamed that horrible thing put talk page....
159568    spitzer umm, theres actual article prostitutio...
159569    and looks like actually put speedy first versi...
159570    " and ... i really think understand. i came id...
Name: comment_text, Length: 159571, dtype: object

### 1.5 Stemming
For stemming we are going to use PorterStemmer <br>

// Detailed comment coming up . . .

In [100]:
from nltk.stem import PorterStemmer

def porter_stemmer(text):
    """
        Stem words in list of tokenized words with PorterStemmer
    """
    stemmer = nltk.PorterStemmer()
    stems = [stemmer.stem(i) for i in text]
    return stems

In [102]:
"""
 
 I am getting unexpected output here. I will just comment it out for now.

"""
#test['comment_text'] = test['comment_text'].apply(lambda x: porter_stemmer(x))
#train['comment_text'] = train['comment_text'].apply(lambda x: porter_stemmer(x))

In [124]:
train['comment_text']

0         explanation edits made username hardcore metal...
1         d'aww! matches background colour seemingly stu...
2         hey man, really trying edit war. guy constantl...
3         " cannot make real suggestions improvement - w...
4                  you, sir, hero. chance remember page on?
                                ...                        
159566    ":::::and second time asking, view completely ...
159567    ashamed horrible thing put talk page. 128.61.1...
159568    spitzer umm, theres actual article prostitutio...
159569    looks like actually put speedy first version d...
159570    " ... really think understand. came idea bad r...
Name: comment_text, Length: 159571, dtype: object

## 2. Näive Bayes Model

## 3. svm

## 4. GRU

## 5. LSTM

## 6. xLNet
Ref. https://github.com/Shivampanwar/Bert-text-classification/blob/master/XLNet/xlnet_experimentation.ipynb
https://towardsdatascience.com/multi-label-text-classification-with-xlnet-b5f5755302df

## 7. Ensembling

## 8. Conclusion